In [52]:
## Notebook utils

import scipy.io

def export_to_matlab(output_file, data):
    scipy.io.savemat(output_file, mdict={'data': data})

In [1]:
## Starting Octave client

from oct2py import Oct2Py

octave = Oct2Py()
octave.addpath('../../matlab/');

In [2]:
## Communication Input data model
# An ad-hoc object with the images to analyze, the points and the algorithm settings. 
# Points: Dictionary with the Point ID as key and a ad-hoc object with PositionX, PositionY and a list of the two images (as PIL.Image.Image) as value.
# Settings.TimeDelta: Time between two images, iin miliseconds.
# Settings.Scale: Image scaling, in pixels per milimeters.
# Settings.WindowSize: Interrogation Window size, default is 32.
# Settings.RoiSize: Region of Interest size, default is None which will be used as the full image.

class InputPIV:
    def __init__(self, points, time_delta, scale, window_size=32, roi_size=None):
        self.points = points
        self.settings = Settings(time_delta, scale, window_size, roi_size)
        

class Settings:
    def __init__(self, time_delta, scale, window_size, roi_size):
        self.time_delta = time_delta
        self.scale = scale
        self.window_size = window_size
        self.roi_size = roi_size
        

class Point:
    def __init__(self, pos_x, pos_y, images):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.images = images


## Communication Output data model
# An ad-hoc object with the following fields: X, Y, U (X velocity), V (Y velocity) and S2N (signal to noise ratio).

class OutputPIV:
    def __init__(self, x, y, u, v, s2n):
        self.x = x
        self.y = y
        self.u = u
        self.v = v
        self.s2n = s2n

In [3]:
# Utils

def first(aList):
    return aList[0]

def last(aList):
    return aList[-1]

In [4]:
## Externals

import numpy as np
from PIL import Image


## Reading images
# Loading images as an IxJ matrix, containing the intensity of each pixel.
#
# Output: 
# Array with the following dimensions: 0 - Image; 1 - Height (Y); 2 - Width (X).

IMAGE_1 = '../images/Image 1a.png'
IMAGE_2 = '../images/Image 1b.png'

def load_images(images_paths=[IMAGE_1, IMAGE_2]):
    images = []
    
    for image in images_paths:
        img = Image.open(image)
        grayscale_image = img.convert("L")
        grayscale_array = np.asarray(grayscale_image)
        images += [np.array(grayscale_array)]
    
    return np.array(images)


def load_fake_images(y=100, x=None, total_images=5, mode='const'):
    if not x:
        x = y
        
    count = 1
    images = []
    for idx in range(total_images):
        if mode == 'rand':
            images += [(np.random.rand(y, x) * 100).astype(np.uint8)]
        elif mode == 'inc':
            images += [np.reshape(np.arange(count, count + y * x), [y, x], order='F')]
            count += y * x
        else:
            images += [np.ones((y, x), np.uint8) * (idx + 1)]
    return np.array(images)

In [5]:
## Externals

import numpy as np


## Single to double frame
# Combines images by 2, returning an array with two frames (one for each image). 
#
#   Input: 5 images with step 1.
#   Output: 4 double-framed images.
#      FrameA:  1  2  3  4
#      FrameB:  2  3  4  5
#
#   Input: 8 images with step 3.
#   Output: 5 doubled-framed images.
#      FrameA:  1  2  3  4  5
#      FrameB:  4  5  6  7  8
#
# This function also crops the image according to the provided Region of Interest (ROI), that must be passed as:
# ROI = [X-start X-end Y-start Y-end], for example: [1 100 1 50].
#
# Output:
# Array with the following dimensions: 0 - Image; 1 - Frame; 2 - Height (Y); 3 - Width (X).

def single_to_double_frame(images, step=1, roi=None):
    total_images = images.shape[0]

    frameA_idx = list(range(0,total_images-step))
    frameB_idx = [idx+1 for idx in frameA_idx]

    height, width = first(images).shape
    mask = np.ones([height, width], np.uint8)

    images_double_framed = []
    for idx in frameA_idx:
        double_frame = [images[frameA_idx[idx]], images[frameB_idx[idx]]]
            
        if roi and len(roi) == 4:
            size_y, size_x = double_frame[0].shape
            min_x, max_x = max(0, roi[0]-1), min(roi[1], size_x)
            min_y, max_y = max(0, roi[2]-1), min(roi[3], size_x)
            
            double_frame[0] = np.array(double_frame[0][min_y:max_y, min_x:max_x])
            double_frame[1] = np.array(double_frame[1][min_y:max_y, min_x:max_x])

        images_double_framed += [double_frame]
            
    return np.array(images_double_framed)

In [6]:
## Externals

import math
import numpy as np
import numpy.matlib as npmb


## Prepare images for PIV
# Determine which indices must be used to create the interrogation windows. 
# It also add a padding dark color to the images.
#
# Output: Indexes for vectors (MinX, MaxX, MinY, MaxY), the padded images and the interrogation window indexes.

def prepare_piv_images(images, window_size, step):
    
    # Calculating vectors.
    min_x = 1 + math.ceil(step)
    min_y = 1 + math.ceil(step)
    size_y, size_x = first(images)[0].shape
    max_x = step * math.floor(size_x / step) - (window_size - 1) + math.ceil(step)
    max_y = step * math.floor(size_y / step) - (window_size - 1) + math.ceil(step)
    vectors_u = math.floor((max_x - min_x)/step + 1)
    vectors_v = math.floor((max_y - min_y)/step + 1)
    
    # Centering image grid.
    pad_x = size_x - max_x
    pad_y = size_y - max_y
    shift_x = max(0, round((pad_x - min_x) / 2))
    shift_y = max(0, round((pad_y - min_y) / 2))
    min_x += shift_x
    min_y += shift_y
    max_x += shift_x
    max_y += shift_y
    
    # Adding a dark padded border to images.
    padded_images = []
    for idx in range(len(images)):
        padded_images += [[]]
        for frame in range(2):
            image = images[idx][frame]
            padded_images[idx] += [np.pad(image, math.ceil(window_size-step), constant_values=image.min())]
        padded_images[idx] = np.array(padded_images[idx])
    padded_images = np.array(padded_images)
    
    # Interrogation window indexes for first frame.
    padded_size_y, padded_size_x = first(padded_images)[0].shape
    min_s0 = npmb.repmat(np.array(np.arange(min_y, max_y + 1, step) - 1)[:, None], 1, vectors_u)
    max_s0 = npmb.repmat(np.array(np.arange(min_x, max_x + 1, step) - 1) * padded_size_y, vectors_v, 1)
    s0 = np.asarray(min_s0 + max_s0).flatten()[..., np.newaxis, np.newaxis].transpose([1, 2, 0])

    min_s1 = npmb.repmat(np.array(np.arange(1, window_size + 1))[:, None], 1, window_size)
    max_s1 = npmb.repmat(np.array(np.arange(1, window_size + 1) - 1) * padded_size_y, window_size, 1)
    s1 = min_s1 + max_s1

    indexes = np.tile(np.asarray(s1)[..., np.newaxis], [1, 1, s0.shape[2]]) + np.tile(s0, [window_size, window_size, 1]) - 1
    
    return min_x, max_x, min_y, max_y, padded_images, indexes

In [7]:
## Externals

import numpy as np


## Cumulative cross correlation
# Averages correlation maps from an image stack.
#
# TODO: This function isn't working properly! Matlab FFT ≠ Numpy FFT.
# Should fix the cross correlation calculation and also check the normalization (different shape expected).
#
# Output: A correlation matrix with the same size as the images input.

NORMALIZED_CORRELATION_RESOLUTION = 2**8
def cumulative_cross_correlation(images, indexes, window_size):
    
    total_correlation = 0
    for idx, image in enumerate(images):
        frame_a = image[0].take(indexes).astype(np.single)
        frame_b = image[1].take(indexes).astype(np.single)
        
        # Calculating cross correlation
        fft_a = np.fft.fft2(frame_a)
        fft_b = np.fft.fft2(frame_b)

        fft_shifting = np.real(np.fft.ifft(np.fft.ifft(np.conj(fft_a) * fft_b, window_size, 1), window_size, 0))
        correlation = np.fft.fftshift(np.fft.fftshift(fft_shifting, 2), 1)
        correlation[correlation < 0] = 0
        
        # Normalizing correlation
        min_corr = np.tile(correlation.min(0).min(0), [correlation.shape[0], correlation.shape[1], 1])
        max_corr = np.tile(correlation.max(0).max(0), [correlation.shape[0], correlation.shape[1], 1])
        norm_corr = (correlation - min_corr) / (max_corr - min_corr) * (NORMALIZED_CORRELATION_RESOLUTION - 1)
    
        total_correlation += norm_corr/len(images)
        
    return total_correlation

In [8]:
## Externals

import numpy as np
import scipy.ndimage


## Vector field determination
# Here it's where magic happens, calculating peaks and doing science stuff to get the proper PIV data.
#
# Output: OutputPIV object

S2N_FILTER = False
DEFAULT_S2N_THRESHOLD = 1
DEFAULT_RES_NORMALIZATION = 255
def vector_field_determination(correlation, int_window, step, min_x, max_x, min_y, max_y):
    
    # Normalize result
    squeezed_min_corr = correlation.min(0).min(0).squeeze()[:, np.newaxis, np.newaxis]
    squeezed_delta_corr = correlation.max(0).max(0).squeeze()[:, np.newaxis, np.newaxis] - squeezed_min_corr
    min_res = np.tile(squeezed_min_corr, [1, correlation.shape[0], correlation.shape[1]]).transpose([1, 2, 0])
    delta_res = np.tile(squeezed_delta_corr, [1, correlation.shape[0], correlation.shape[1]]).transpose([1, 2, 0])
    corr = ((correlation - min_res) / delta_res) * DEFAULT_RES_NORMALIZATION
    
    # Find peaks and S2N
    x1, y1, indexes1, x2, y2, indexes2, s2n = find_all_displacements(corr)
    
    # Sub-pixel determination
    pixel_offset = 1 if (int_window % 2 == 0) else 0.5
    vector = sub_pixel_gaussian(corr, int_window, x1, y1, indexes1, pixel_offset)
    
    # Create data
    x_range = np.arange(min_x, max_x + 1, step)
    y_range = np.arange(min_y, max_y + 1, step)
    output_x = np.tile(x_range + int_window / 2, [len(y_range), 1])
    output_y = np.tile(y_range[:, None] + int_window / 2, [1, len(x_range)])
    vector = np.reshape(vector, np.append(np.array(output_x.transpose().shape), 2), order='F').transpose([1, 0, 2])

    # Signal to noise filter
    s2n = s2n[np.reshape(np.array(range(output_x.size)), output_x.transpose().shape, order='F').transpose()]
    if S2N_FILTER:
        vector[:,:,0] = vector[:,:,0] * (s2n > DEFAULT_S2N_THRESHOLD)
        vector[:,:,1] = vector[:,:,1] * (s2n > DEFAULT_S2N_THRESHOLD)
    
    output_u = vector[:,:,0]
    output_v = vector[:,:,1]

    output_x -= int_window/2
    output_y -= int_window/2

    return OutputPIV(output_x, output_y, output_u, output_v, s2n)
    
    
## Gaussian sub-pixel mode
# No f*cking clue what this does. Crazy math shit.
#
# Output: A vector with a sub-pixel deviation - Maybe? I'm not sure. Its dimensions are Number-of-Correlations by 2. 

def sub_pixel_gaussian(correlation, int_window, x, y, indexes, pixel_offset):
    z = np.array(range(indexes.shape[0])).transpose()
    
    xi = np.nonzero(np.logical_not(np.logical_and(
        # Adjusting -1 to -2 according to Matlab/Python mapping.
        np.logical_and(x <= correlation.shape[1] - 2, y <= correlation.shape[0] - 2),
        np.logical_and(x >= 2, y >= 2)
    )))[0]

    x = np.delete(x, xi)
    y = np.delete(y, xi)
    z = np.delete(z, xi)
    x_max = correlation.shape[1]
    vector = np.ones((correlation.shape[2], 2)) * np.nan

    if len(x) > 0:
        ip = np.ravel_multi_index(np.array([x, y, z]), correlation.shape, order='F')
        flattened_correlation = correlation.flatten(order='F')

        f0 = np.log(flattened_correlation[ip])
        f1 = np.log(flattened_correlation[ip - 1])
        f2 = np.log(flattened_correlation[ip + 1])
        peak_y = y + (f1 - f2) / (2 * f1 - 4 * f0 + 2 * f2)

        f1 = np.log(flattened_correlation[ip - x_max])
        f2 = np.log(flattened_correlation[ip + x_max])
        peak_x = y + (f1 - f2) / (2 * f1 - 4 * f0 + 2 * f2)
    
        sub_pixel_x = peak_x - (int_window / 2) - pixel_offset
        sub_pixel_y = peak_y - (int_window / 2) - pixel_offset
    
        vector[z, :] = np.array([sub_pixel_x, sub_pixel_y]).transpose()
    
    return vector

    
## Find all displacements
# Find all integer pixel displacement in a stack of correlation windows.
#
# Output: Horizontal and vertical indexes of the first and second maximum for each slice of correlation in the third
# dimension (PeakX1, PeackY1, PeakX2, PeakY2), the absolute indexes of the correlation maximums (Idx1, Idx2) and the
# ratio between the first and second peack (S2N) - 0 indicates non confiable results.

def find_all_displacements(correlation):
    corr_size = correlation.shape[0]
    
    # Finding first peak
    peak1_val, peak1_x, peak1_y, peak_indexes1, peak_positions1 = find_peaks(correlation)

    # Finding second peak (1 extra point from Matlab size)
    filter_size = 10 if corr_size >= 64 else 5 if corr_size >= 32 else 4
    filtered = scipy.ndimage.correlate(peak_positions1, np.ones([filter_size, filter_size, 1]), mode='constant')
    correlation = (1 - filtered) * correlation
    peak2_val, peak2_x, peak2_y, peak_indexes2, _ = find_peaks(correlation)

    # Calculating Signal to Noise ratio
    signal_to_noise = np.zeros([peak1_val.shape[0]])
    signal_to_noise[peak2_val != 0] = peak1_val[peak2_val != 0] / peak2_val[peak2_val != 0]

    # Maximum at a border usually indicates that MAX took the first one it found, so we should put a bad S2N, like 0.
    signal_to_noise[peak1_y == 0] = 0
    signal_to_noise[peak1_x == 0] = 0
    signal_to_noise[peak1_y == (corr_size - 1)] = 0
    signal_to_noise[peak1_x == (corr_size - 1)] = 0
    signal_to_noise[peak2_y == 0] = 0
    signal_to_noise[peak2_x == 0] = 0
    signal_to_noise[peak2_y == (corr_size - 1)] = 0
    signal_to_noise[peak2_x == (corr_size - 1)] = 0
    
    return peak1_x, peak1_y, peak_indexes2, peak2_x, peak2_y, peak_indexes2, signal_to_noise
    
    
## Find peaks
# Find max values for each correlation.
#
# Output: The MAX peak, its coordinates (X and Y) and the indexes.
    
def find_peaks(correlation):
    corr_size = correlation.shape[0]
    corr_numbers = correlation.shape[2]
    max_peak = correlation.max(0).max(0)
    max_positions = correlation == np.tile(max_peak[np.newaxis, np.newaxis, ...], [corr_size, corr_size, 1])
    max_indexes = np.where(max_positions.transpose(2, 1, 0).flatten())[0]
    peak_y, peak_x, peak_z = np.unravel_index(max_indexes, (corr_size, corr_size, corr_numbers), order='F')

    # If two elements equals to the max we should check if they are in the same layer and take the first one.
    # Surely the second one will be the second highest peak. Anyway this would be a bad vector.
    unique_max_indexes = np.unique(peak_z)
    max_indexes = max_indexes[unique_max_indexes]
    peak_x = peak_x[unique_max_indexes]
    peak_y = peak_y[unique_max_indexes]
    
    return max_peak, peak_x, peak_y, max_indexes, max_positions

In [9]:
## Externals

import numpy as np
import scipy.sparse
import scipy.sparse.linalg


## Filter fields (WIP)
# Applies different filters on the vector fields.
#
# Output: OutputPIV object, with filtered data.

B = 1
EPSILON = 0.02
DEFAULT_THRESH = 1.5
DEFAULT_STD_THRESHOLD = 4
def filter_fields(data, std_threashold=DEFAULT_STD_THRESHOLD):
    # Filter 1: Threshold on signal to noise.  
    data.u = remove_nans(data.u)
    data.v = remove_nans(data.v)
    
    # Filter 2:
    mean_u = np.mean(data.u)
    mean_v = np.mean(data.v)
    std_u = np.std(data.u, ddof=1)
    std_v = np.std(data.v, ddof=1)
    min_u = mean_u - std_threashold * std_u
    max_u = mean_u + std_threashold * std_u
    min_v = mean_v - std_threashold * std_v
    max_v = mean_v + std_threashold * std_u
    data.u[data.u < min_u] = np.nan
    data.u[data.u > max_u] = np.nan
    data.v[data.v < min_v] = np.nan
    data.v[data.v > max_v] = np.nan
    
    # Filter 3:
    size_y, size_x = data.u.shape
    normal_fluctuation = np.zeros(shape=(size_y, size_x, 2))
    for it in range(2):
        velocity_comparator = data.u if it == 0 else data.v
        neighbors = np.empty(shape=(size_y - 2, size_x - 2, 2 * B + 1, 2 * B + 1))
    
        for ii in range(-B, B + 1):
            for jj in range(-B, B + 1):
                ii_start = 1 + B - 1 + ii
                ii_end = -B + ii if -B + ii < 0 else None
                jj_start = 1 + B - 1 + jj
                jj_end = -B + jj if -B + jj < 0 else None
            
                ii_neighbors = ii + 2 * B - 1
                jj_neighbors = jj + 2 * B - 1
            
                neighbors[:, :, ii_neighbors, jj_neighbors] = velocity_comparator[ii_start:ii_end, jj_start:jj_end]
    
        first_neighbors = np.arange((2 * B + 1) * B + B)
        last_neighbors = np.arange((2 * B + 1) * B + B + 1, (2 * B + 1)**2)
        neighbors_column = np.reshape(neighbors, [neighbors.shape[0], neighbors.shape[1], (2 * B + 1)**2], order='F')
        neighbors_column2 = neighbors_column[:, :, np.append(first_neighbors, last_neighbors)].transpose([2, 0, 1])
    
        median = np.median(neighbors_column2, axis=0).transpose()
        velocity_comparator2 = velocity_comparator[B:-B, B:-B]
        fluctuation = velocity_comparator2 - median.transpose()
        result = neighbors_column2 - np.tile(median, [(2 * B + 1)**2 - 1, 1, 1]).transpose([0, 2, 1])
    
        median_result = np.median(np.abs(result), axis=0)
        normal_fluctuation[B:-B, B:-B, it] = np.abs(fluctuation / (median_result + EPSILON))
    
    info = np.sqrt(normal_fluctuation[:, :, 0] ** 2 + normal_fluctuation[:, :, 1] ** 2) > DEFAULT_THRESH
    data.u[info] = np.nan
    data.v[info] = np.nan
                
    # Inpaint NANs
    data.u = inpaint_nans(data.u)
    data.v = inpaint_nans(data.v)
    
    # Filter 4:
    try:
        
        # Trying to apply the smooth predictor.
        data.u = smooth(data.u)
        data.v = smooth(data.v)
        
    except:
        
        # Applying Gaussian filter instead.
        gfilter = gaussian_filter(5, 1)
        data.u = scipy.ndimage.convolve(data.u, gfilter, mode='nearest')
        data.v = scipy.ndimage.convolve(data.v, gfilter, mode='nearest')
    
    return data
    

## Remove NANs
# Replace all the NANs from a data vector with a custom interpolation calculated with its values.
#
# Output: A matrix with the same dimensions ang items as the input, but with NANs replaced.

DEFAULT_PATCH_SIZE = 1
def remove_nans(data, patch_size=DEFAULT_PATCH_SIZE):
    both_nan_indexes = list(zip(*np.where(np.isnan(data))))
    size_y, size_x = data.shape

    fixed_data = data.copy()
    for y_idx, x_idx in both_nan_indexes:
        sample = data[
            max(0, y_idx - patch_size):min(size_y, y_idx + patch_size + 1), 
            max(0, x_idx - patch_size):min(size_x, x_idx + patch_size + 1)
        ]

        sample = sample[~np.isnan(sample)]
        new_data = np.median(sample) if sample.size > 0 else 0

        fixed_data[y_idx, x_idx] = new_data

    return fixed_data


# Inpaint NANs
# Solves approximation to one of several pdes to interpolate and extrapolate holes in an array.
# It uses a spring metaphor, assuming they (with a nominal length of zero) connect each node with every neighbor 
# (horizontally, vertically and diagonally). Since each node tries to be like its neighbors, extrapolation is as a 
# constant function where this is consistent with the neighboring nodes.
#
# Output: A matrix with the same dimensions ang items as the input, but with NANs replaced.

DEFAULT_SPRING_ITERATIONS = 4
def inpaint_nans(data, iterations=DEFAULT_SPRING_ITERATIONS):
    size_y, size_x = data.shape
    flattened = data.flatten(order='F')

    # List the nodes which are known, and which will be interpolated.
    nan_indexes = np.where(np.isnan(flattened))[0]
    known_indexes = np.where(~np.isnan(flattened))[0]

    # Get total NANs overall.
    nan_count = nan_indexes.size

    # Convert NAN indexes to [Row, Column] form.
    indexes_y, indexes_x = np.unravel_index(nan_indexes, (size_y, size_x), order='F')

    # All forms of index in one array: 0 - Unrolled ; 1 - Row ; 2 - Column
    nan_list = np.array([nan_indexes, indexes_y, indexes_x]).transpose() + 1

    # Spring analogy - interpolating operator.
    # List of all springs between a node and a horizontal or vertical neighbor.
    hv_list = np.array([[-1, -1, 0], [1, 1, 0], [-size_y, 0, -1], [size_y, 0, 1]])
    hv_springs = np.empty((0, 2))

    for it in range(iterations):
        hvs = nan_list + np.tile(hv_list[it, :], (nan_count, 1))
        k = np.logical_and(
            np.logical_and(hvs[:, 1] >= 1, hvs[:, 1] <= size_y),
            np.logical_and(hvs[:, 2] >= 1, hvs[:, 2] <= size_x)
        )
        hv_springs = np.append(hv_springs, np.array([nan_list[k, 0], hvs[k, 0]]).transpose(), axis=0)
    
    # Delete replicate springs    
    hv_springs.sort(axis=1)
    hv_springs = np.unique(hv_springs, axis=0) - 1

    # Build sparse matrix of connections.
    # Springs connecting diagonal neighbors are weaker than the horizontal and vertical ones.
    nhv = hv_springs.shape[0]
    I, V = np.tile(np.arange(0, nhv)[:, None], (1, 2)).flatten(), np.tile([1, -1], (nhv, 1)).flatten()
    springs = scipy.sparse.csr_matrix((V, (I, hv_springs.flatten())), shape=(nhv, data.size))
    springs.eliminate_zeros()

    # Eliminate knowns
    rhs = springs[:, known_indexes] * flattened[known_indexes] * -1

    # Solve problem
    output = flattened
    solution, _, _, _, _, _, _, _, _ ,_ = scipy.sparse.linalg.lsqr(springs[:, nan_indexes], rhs) 
    output[nan_indexes] = solution

    return np.reshape(output, (size_x, size_y)).transpose()


## Smooth predictor (WIP)
# Fast, automatized and robust discretized spline smoothing for data of arbitrary dimension.
# Automatically smoothes the uniformly-sampled input array. It can be any N-D noisy array (time series, images, 
# 3D data, ...). Non finite data (NaN or Inf) are treated as missing values.
#
# Output: A matrix with the same dimensions ang items as the input, but with NANs replaced.

def smooth(data):
    return octave.smoothn(data)

    
## Gaussian filter
# Returns a Gaussian filter with the same implementation as Matlab.
#
# Output: A matrix that works as a Gaussian filter.

def gaussian_filter(size=3, sigma=0.5):
    m,n = [(ss-1.)/2. for ss in (size, size)]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

In [10]:
## Calculate PIV
# Generate the PIV data from the images loaded with the input parameters.
#
# Output: OutputPIV object

DEFAULT_OVERLAP = 0.5
def PIV(images, int_window, overlap=DEFAULT_OVERLAP):
    step = round(int_window * overlap)
    min_x, max_x, min_y, max_y, padded_images, indexes = prepare_piv_images(images, int_window, step)
    cross_correlation = cumulative_cross_correlation(padded_images, indexes, int_window)
    raw_piv_data = vector_field_determination(cross_correlation, int_window, step, min_x, max_x, min_y, max_y)
    filtered_piv_data = filter_fields(raw_piv_data)

    filtered_piv_data.x = filtered_piv_data.x.transpose()
    filtered_piv_data.y = filtered_piv_data.y.transpose()
    filtered_piv_data.u = filtered_piv_data.u.transpose()
    filtered_piv_data.v = filtered_piv_data.v.transpose()
    filtered_piv_data.s2n = filtered_piv_data.s2n.transpose()
    
    return filtered_piv_data

In [11]:
## Externals

import numpy as np


## Communication Exceptions
# Exception thrown when some parameters weren't passed as expected.
        
class InvalidParametersError(Exception):
    pass


## Prepare output
# Get the velocity for the desired point. If it is not possible, it will get it for the closest point.
#
# Output: OutputPIV object

def prepare_output(center_x, center_y, piv_data):
    idx_x = (np.abs(piv_data.x[:,1] - center_x)).argmin()
    idx_y = (np.abs(piv_data.y[1,:] - center_y)).argmin()

    position_x = int(piv_data.x[idx_x,1]) + 1
    position_y = int(piv_data.y[1,idx_y]) + 1
    velocity_x = piv_data.u[idx_x,idx_y]
    velocity_y = piv_data.v[idx_x,idx_y]
    signal_to_noise = piv_data.s2n[idx_x,idx_y]
    
    return OutputPIV(position_x, position_y, velocity_x, velocity_y, signal_to_noise)


## Entrypoint
# Retrieve the images, prepare them and calculate the PIV computation.
#
# Output: OutputPIV object

DEFAULT_INTERROGATION_WINDOW = 32
def calculate_piv(frontend_data):
    results = {}
    settings = frontend_data.settings
    
    # TODO: Check if this could be parallelized to increase performance.
    for point_id, point_data in frontend_data.points.items():

        double_framed_images = single_to_double_frame(point_data.images)
        if double_framed_images.size <= 2:
            raise InvalidParametersError(f'Not enough images passed for point {point_id}')
            
        shift_x = 0
        shift_y = 0
        if settings.roi_size is not None:
            roi_shift = int(settings.roi_size / 2)
            shift_x = point_data.pos_x - roi_shift
            shift_y = point_data.pos_y - roi_shift
        
        piv_data = PIV(double_framed_images, settings.window_size)
        piv_data.x = piv_data.x * settings.scale + shift_x
        piv_data.y = piv_data.y * settings.scale + shift_y
        piv_data.u = piv_data.u * settings.scale / settings.time_delta
        piv_data.v = piv_data.v * settings.scale / settings.time_delta
        
        point_results = prepare_output(point_data.pos_x - 1, point_data.pos_y - 1, piv_data)
        results[point_id] = point_results
    
    return results

In [12]:
inputs = load_images()
window_size, overlap = 32, 0.5

step = round(window_size * overlap)
double_framed = single_to_double_frame(inputs)
min_x, max_x, min_y, max_y, images, indexes = prepare_piv_images(double_framed, window_size, step)

## CUMULATIVE CROSS CORRELATION

NORMALIZED_CORRELATION_RESOLUTION = 2**8
    
total_correlation = 0
for idx, image in enumerate(images):
    frame_a = image[0].take(indexes).astype(np.single)
    frame_b = image[1].take(indexes).astype(np.single)
        
    # Calculating cross correlation
    fft_a = np.fft.fft2(frame_a)
    fft_b = np.fft.fft2(frame_b)

    fft_shifting = np.real(np.fft.ifft(np.fft.ifft(np.conj(fft_a) * fft_b, window_size, 1), window_size, 0))
    correlation = np.fft.fftshift(np.fft.fftshift(fft_shifting, 2), 1)
    correlation[correlation < 0] = 0
        
    # Normalizing correlation
    min_corr = np.tile(correlation.min(0).min(0), [correlation.shape[0], correlation.shape[1], 1])
    max_corr = np.tile(correlation.max(0).max(0), [correlation.shape[0], correlation.shape[1], 1])
    norm_corr = (correlation - min_corr) / (max_corr - min_corr) * (NORMALIZED_CORRELATION_RESOLUTION - 1)
    
    total_correlation += norm_corr/len(images)

In [102]:
inputs = load_images()
window_size, overlap = 32, 0.5

step = round(window_size * overlap)
double_framed = single_to_double_frame(inputs)
min_x, max_x, min_y, max_y, images, indexes = prepare_piv_images(double_framed, window_size, step)

## CUMULATIVE CROSS CORRELATION
    
total_correlation = 0
for idx, image in enumerate(images):
    frame_a = image[0].flatten(order='F').take(indexes).astype(np.single)
    frame_b = image[1].flatten(order='F').take(indexes).astype(np.single)
        
    # Calculating cross correlation
    correlation = octave.correlate(frame_a, frame_b, window_size)
        
    # Normalizing correlation
    min_corr = np.tile(correlation.min(0).min(0), [correlation.shape[0], correlation.shape[1], 1])
    max_corr = np.tile(correlation.max(0).max(0), [correlation.shape[0], correlation.shape[1], 1])
    norm_corr = (correlation - min_corr) / (max_corr - min_corr) * (NORMALIZED_CORRELATION_RESOLUTION - 1)
    
    total_correlation += norm_corr/len(images)

In [107]:
total_correlation[0:5,0:5,0], total_correlation2[0:5,0:5,0]

(array([[150.54735 , 163.85245 , 175.57785 , 183.57521 , 179.56801 ],
        [130.88586 , 143.8335  , 151.7764  , 158.77962 , 155.92657 ],
        [114.92517 , 125.207016, 134.20534 , 137.34096 , 136.87793 ],
        [109.70593 , 116.26657 , 121.80243 , 123.940506, 125.8709  ],
        [112.78027 , 116.784065, 115.0205  , 113.21266 , 116.37892 ]],
       dtype=float32),
 array([[  0.        ,   0.        ,  29.02635984,  44.20187396,
           0.        ],
        [  0.        ,  12.39216791,   0.        ,   7.84717792,
           0.        ],
        [ 31.61917328,  22.13879916, 112.91599092,  85.16986718,
           0.        ],
        [  0.        ,   0.96791558,   0.        ,   2.45109429,
           0.        ],
        [  0.        ,   0.        ,  23.32560112,  16.4664921 ,
           0.        ]]))